In [120]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath('')))

from Database import PanelDataBase
from Vector import Vector

import numpy as np
import pandas as pd

from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [121]:
db = PanelDataBase()
panels = db.panel_list[::2]


Database target directory:  /home/kolya/code/diploma/Data


In [122]:
panel = panels[0]
train_dframe = pd.concat([db[panel].initial_data,db[panel].second_run_data]) 
train_features = train_dframe.loc[:, ["x_light", "y_light"]]
train_X = train_dframe.loc[:, "X"]
train_Y = train_dframe.loc[:, "Y"]

test_dframe = db[panel].second_run_data.copy()
test_features = test_dframe.loc[:, ["x_light", "y_light"]]
test_X = test_dframe.loc[:, "X"]
test_Y = test_dframe.loc[:, "Y"]


In [123]:
mlp_X = make_pipeline(StandardScaler(), 
                      MLPRegressor(hidden_layer_sizes=(32,8,4), 
                                   solver='lbfgs', 
                                   alpha=0.0001, 
                                   activation='tanh', 
                                   max_iter=2000, 
                                   early_stopping=True, 
                                   random_state=21, 
                                   learning_rate_init=0.01, 
                                   learning_rate='adaptive'))
mlp_Y = make_pipeline(StandardScaler(),
                      MLPRegressor(hidden_layer_sizes=(32,8,4), 
                                   solver='lbfgs', 
                                   alpha=0.0001, 
                                   activation='tanh', 
                                   max_iter=2000, 
                                   early_stopping=True, 
                                   random_state=21, 
                                   learning_rate_init=0.01, 
                                   learning_rate='adaptive'))

In [124]:
mlp_X.fit(train_features, train_X)
mlp_Y.fit(train_features, train_Y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('mlpregressor',
                 MLPRegressor(activation='tanh', early_stopping=True,
                              hidden_layer_sizes=(32, 8, 4),
                              learning_rate='adaptive', learning_rate_init=0.01,
                              max_iter=2000, random_state=21,
                              solver='lbfgs'))])

In [125]:
pred_X = mlp_X.predict(test_features)
pred_Y = mlp_Y.predict(test_features)

In [126]:
errors = [Vector.angle_between_vectors(Vector(r_X, r_Y), Vector(p_X, p_Y)) for r_X, r_Y, p_X, p_Y in zip(test_X, test_Y, pred_X, pred_Y)]
errors = pd.Series(errors).dropna()

In [127]:
print(np.mean(errors))
print(np.std(errors))

0.5489148027576157
0.3910072071793941
